# Ch.07 ScrapyとMySQL

<https://sugiaki1989.gitbook.io/scrapy-note/chapter07_mysql>

[Books to scrape](http://books.toscrape.com/)

- MySQLをSQLiteに変更する

## プロジェクトの作成

```bash
scrapy startproject sample_books_sqlite

cd sample_books_sqlite

scrapy genspider books_spider_sqlite books.toscrape.com
```

## アイテムの定義（SQLite）

- items.py
- Itemは、スクレイピングしたデータを格納しておくためのオブジェクト
- Itemに格納して、SQLiteに保存するためのパイプラインにデータを流す

```python
# items.py
import scrapy

class BooksSqliteItem(scrapy.Item):
    title = scrapy.Field()
    price = scrapy.Field()
    detail_page_url = scrapy.Field()
```

## クローラーの設計

```bash
scrapy shell https://books.toscrape.com
```

```bash
>>> response.xpath('.//*[@class="product_pod"]')
[<Selector query='.//*[@class="product_pod"]' data='<article class="product_pod">\n       ...'>, <Selector query='.//*[@class="product_pod"]' data='<article class="product_pod">\n       ...'>, <Selector query='.//*[@class="product_pod"]' data='<article class="product_pod">\n       ...'>, <Selector query='.//*[@class="product_pod"]' data='<article class="product_pod">\n       ...'>, <Selector query='.//*[@class="product_pod"]' data='<article class="product_pod">\n       ...'>, <Selector query='.//*[@class="product_pod"]' data='<article class="product_pod">\n       ...'>, <Selector query='.//*[@class="product_pod"]' data='<article class="product_pod">\n       ...'>, <Selector query='.//*[@class="product_pod"]' data='<article class="product_pod">\n       ...'>, <Selector query='.//*[@class="product_pod"]' data='<article class="product_pod">\n       ...'>, <Selector query='.//*[@class="product_pod"]' data='<article class="product_pod">\n       ...'>, <Selector query='.//*[@class="product_pod"]' data='<article class="product_pod">\n       ...'>, <Selector query='.//*[@class="product_pod"]' data='<article class="product_pod">\n       ...'>, <Selector query='.//*[@class="product_pod"]' data='<article class="product_pod">\n       ...'>, <Selector query='.//*[@class="product_pod"]' data='<article class="product_pod">\n       ...'>, <Selector query='.//*[@class="product_pod"]' data='<article class="product_pod">\n       ...'>, <Selector query='.//*[@class="product_pod"]' data='<article class="product_pod">\n       ...'>, <Selector query='.//*[@class="product_pod"]' data='<article class="product_pod">\n       ...'>, <Selector query='.//*[@class="product_pod"]' data='<article class="product_pod">\n       ...'>, <Selector query='.//*[@class="product_pod"]' data='<article class="product_pod">\n       ...'>, <Selector query='.//*[@class="product_pod"]' data='<article class="product_pod">\n       ...'>]
```

```python
# books_spider_sqlite.py
from scrapy import Spider
from scrapy.http import Request
from sample_books_sqlite.items import BooksSqliteItem


class BooksSpiderSqliteSpider(Spider):
    name = "books_spider_sqlite"
    allowed_domains = ["books.toscrape.com"]
    start_urls = ["https://books.toscrape.com"]

    def parse(self, response):
        books = response.xpath('.//*[@class="product_pod"]')
        for book in books:
            item = BooksSqliteItem()

            item["title"] = book.xpath('.//h3/a/@title').get()
            item["price"] = book.xpath('.//*[@class="price_color"]/text()').get()
            img_url = book.xpath('.//*[class="image_container"]/a/@href').get()
            item["detail_page_url"] = response.urljoin(img_url)

            yield item

        # If there is a next button on this page, move the crawler
        # このページに「Next」ボタンがある場合は、クローラーを移動させる。
        next_page_url = response.xpath('//a[text()="next"]/@href').get()
        abs_next_page_url = response.urljoin(next_page_url)
        if abs_next_page_url is not None:
            yield Request(abs_next_page_url, callback=self.parse)
```

## パイプラインの設計

- pipelines.py
- Itemに保存されているデータをSQLiteにインサートするためのパイプライン
  - データベースにコネクションを作る関数
  - データベースに重複チェックしてインサートする関数
  - データベースのコネクションをクロースする関数

```python
# pipelines.py
import sqlite3


class BooksSqlitePipeline:
    def open_spider(self, spider) -> None:
        # Create/Connect to database
        self.connection = sqlite3.connect("book_online.db")
        # Create cursor
        self.cursor = self.connection.cursor()
        # booksテーブルが存在しない場合は作成する
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS books(
                id INTEGER PRIMARY KEY,
                title TEXT,
                price TEXT,
                detail_page_url TEXT,
            )
        """)

    def process_item(self, item, spider):
        # titleがすでにテーブルにあるかどうかをチェックする
        self.cursor.execute("select * from books where title = ?", (item["title"],))
        result = self.cursor.fetchone()

        # テーブルにある場合は、ログメッセージを作成する
        if result:
            spider.logger.warn("Item already in table: %s" % item["title"])
        # テーブルにtitleがない場合、データを挿入する
        else:
            # insertステートメントの定義
            self.cursor.execute(
                """INSERT INTO books (title, price, detail_page_url) VALUES (?, ?, ?)""",
                (
                    item["title"],
                    item["price"],
                    item["detail_page_url"],
                )
            )
            # テーブルへのデータ挿入の実行
            self.connection.commit()

        return item

    def close_spider(self, spider):
        self.connection.close()
```

## 環境設定

- settings.py
- パイプラインを有効にする

```python
# settings.py
# Configure item pipelines
# See https://docs.scrapy.org/en/latest/topics/item-pipeline.html
ITEM_PIPELINES = {
   "sample_books_sqlite.pipelines.BooksSqlitePipeline": 800,
}
```

## クローラーの実行

```bash
scrapy crawl books_spider_sqlite
```